In [2]:
# Installation of PM4Py Python package
#Source: https://pm4py.fit.fraunhofer.de/install-page
!pip install pm4py

     |████████████████████████████████| 1.5MB 8.5MB/s 
     |████████████████████████████████| 40.6MB 76kB/s 
  Created wheel for stringdist: filename=StringDist-1.0.9-cp37-cp37m-linux_x86_64.whl size=23558 sha256=e3fa055eb70b3313e2d9b0765b81913e18222d9a5a0ea30766dd8b11d5cbec77
  Stored in directory: /root/.cache/pip/wheels/c3/1b/bb/bf0de4d64d8ca38759811fe3353e441f12feb606fb3d1d2e11
Successfully built stringdist


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# Import dependencies and set the filepath
import pm4py
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px         # needed for plotly express in general
import plotly.graph_objects as go   # needed for plotly graph objects e.g. pie

In [24]:
# Set the file path
# Please upload the local file sent via email to colab before running this
file_path = '/content/Hospital Billing - Event Log.xes' # if using the original file
#file_path = '/content/log.xes' # if renamed

In [25]:
# Read the XES file and create the log
event_log = pm4py.read_xes(file_path)

XMLSyntaxError: ignored

In [22]:
def import_xes(file_path):
  event_log = pm4py.read_xes(file_path)
  start_activities = pm4py.get_start_activities(event_log)
  end_activities = pm4py.get_end_activities(event_log)
print("Start activities: {}\nEnd activities: {}".format(start_activities, end_activities))

NameError: ignored

In [21]:
# Obtaining a Process Model
# event_log = pm4py.read_xes('C:/Users/demo/Downloads/running-example.xes')

process_tree = pm4py.discover_process_tree_inductive(event_log) # discover_tree_inductive is deprecated
bpmn_model = pm4py.convert_to_bpmn(process_tree)
pm4py.view_bpmn(bpmn_model)

NameError: ignored

In [ ]:
#obtain process map 
dfg, start_activities, end_activities = pm4py.discover_dfg(event_log)
pm4py.view_dfg(dfg, start_activities, end_activities)

In [ ]:
#heuristics map
map = pm4py.discover_heuristics_net(event_log)
pm4py.view_heuristics_net(map)

In [ ]:
# View Process Tree
process_tree = pm4py.discover_process_tree_inductive(event_log) # discover_tree_inductive is deprecated
pm4py.view_process_tree(process_tree)

In [19]:
print("Start activities: {}\nEnd activities: {}".format(start_activities, end_activities))

NameError: ignored

## Miner Algorithms
Please refer to the documentation here:
https://pm4py.fit.fraunhofer.de/documentation#item-3-1

In [ ]:
# Alpha Miner implementation

from pm4py.algo.discovery.alpha import algorithm as alpha_miner
net, initial_marking, final_marking = alpha_miner.apply(event_log)

# Inductive Miner implementation
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
net, initial_marking, final_marking = inductive_miner.apply(event_log)

## Visualisations

In [ ]:
# Visualisation of IM

from pm4py.visualization.process_tree import visualizer as pt_visualizer
tree = inductive_miner.apply_tree(event_log)

gviz = pt_visualizer.apply(tree)
pt_visualizer.view(gviz)

'/tmp/tmpn43cpox3.gv.png'

In [ ]:
# Converting process tree in petri net

from pm4py.objects.conversion.process_tree import converter as pt_converter
net, initial_marking, final_marking = pt_converter.apply(tree, variant=pt_converter.Variants.TO_PETRI_NET)
gviz = pt_visualizer.apply(tree)
pt_visualizer.view(gviz)

'/tmp/tmpgzmi0vfv.gv.png'

In [ ]:
# Dotted chart
pm4py.view_dotted_chart(event_log, format="svg")